In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from copy import copy

class X:
    _ignore_attr=['dict']

    def __init__(_i) -> None:
        
        # for k,v in _i.__class__.__dict__.items():
        #     if k.startswith('_') or k in _i._ignore_attr or isinstance(v, property):
        #         continue
        #     _i.__dict__[k] = copy(v)
        # for k,v in _i.__dict__.items():
        #     if k.startswith('_') or k in _i._ignore_attr:
        #         continue
        #     if isinstance(v, property):
        #         _i.__dict__[k] = copy(v)

        # print(_i)
        pass

    @property
    def dict(_i, _d={}):
        cls_d = _i.__class__.__dict__
        for k,v in cls_d.items():
            if k.startswith('_'):
                continue
            _i.__dict__[k] = getattr(_i, k)
        return _i.__dict__


class Y(X):
    son = True
    @property
    def z(self,):
        return 1
    
    

class B:
    a = 1
    def __init__(self):
        self.y = Y()
        pass

    @property
    def dict(_i):
        for k,v in _i.__class__.__dict__.items():
            if k.startswith('_') or k=='dict':
                continue
            _i.__dict__[k] = copy(v)
            print(issubclass(type(v), X), v, getattr(_i, k))
        
        for k,v in _i.__dict__.items():
            if k.startswith('_') or k=='dict':
                continue
            if isinstance(v, X):
                print(k, v.dict)
            _i.__dict__[k] = copy(v)
            print(issubclass(type(v), X), v, getattr(_i, k))
        return _i.__dict__


# A subclass is also an instance


b = B()
# print(dir(b))
# print(b.y.__class__.__dict__)
# print(b.__class__.__dict__)

# b = Y()
# print(dir(b))
print(b.__dict__)
print(b.__class__.__dict__)
# Class
# Instance
# Subclass
b.y

print(issubclass(type(b.y), X))
b.dict


In [30]:
from copy import copy

class X:
    _ignore_attr=['dict']

    @property
    def dict(_i, _d={}):
        cls_d = _i.__class__.__dict__
        for k,v in cls_d.items():
            if k.startswith('_'):
                continue
            _i.__dict__[k] = getattr(_i, k)
        return _i.__dict__

class Y(X):
    son = True
    @property
    def z(self,):
        return 1
    
class B:
    a = 1
    y = Y

    def __init__(self):
        # self.y = Y()
        pass

    @property
    def dict(_i):
        for k,v in _i.__class__.__dict__.items():
            if k.startswith('_') or k=='dict':
                continue
            _i.__dict__[k] = copy(v)
        
        for k,v in _i.__dict__.items():
            if k.startswith('_') or k=='dict':
                continue
            if isinstance(v, X):
                v = v.dict
            _i.__dict__[k] = copy(v)
        return _i.__dict__


# A subclass is also an instance
b = B()
print(b.__dict__)
print(b.__class__.__dict__)
print(b.y.__dict__)
print(b.y.__class__.__dict__)
print(b.y, issubclass(type(b.y), X), isinstance(b.y, X))
b.dict
b.a
b.a = 2
b.a 
B.a

{}
{'__module__': '__main__', 'a': 1, 'y': <class '__main__.Y'>, '__init__': <function B.__init__ at 0x7fd4ca811c60>, 'dict': <property object at 0x7fd4ca846110>, '__dict__': <attribute '__dict__' of 'B' objects>, '__weakref__': <attribute '__weakref__' of 'B' objects>, '__doc__': None}
{'__module__': '__main__', 'son': True, 'z': <property object at 0x7fd4ca845f80>, '__doc__': None}
{'__new__': <built-in method __new__ of type object at 0x745520>, '__repr__': <slot wrapper '__repr__' of 'type' objects>, '__call__': <slot wrapper '__call__' of 'type' objects>, '__getattribute__': <slot wrapper '__getattribute__' of 'type' objects>, '__setattr__': <slot wrapper '__setattr__' of 'type' objects>, '__delattr__': <slot wrapper '__delattr__' of 'type' objects>, '__init__': <slot wrapper '__init__' of 'type' objects>, '__or__': <slot wrapper '__or__' of 'type' objects>, '__ror__': <slot wrapper '__ror__' of 'type' objects>, 'mro': <method 'mro' of 'type' objects>, '__subclasses__': <method '_

1

In [3]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from jax.config import config
config.update('jax_disable_jit', True)

from pathlib import Path
from typing import Callable
from functools import partial
import wandb

import jax
from jax import numpy as jnp
from jax import random as rnd
from flax.training.train_state import TrainState
from flax.training import common_utils
from flax import linen as nn, jax_utils
import optax

from pyfig import Pyfig
from hwat import create_masks, logabssumdet
from utils import flat_any

2022-11-28 00:02:36.790278: W external/org_tensorflow/tensorflow/compiler/xla/service/gpu/nvptx_compiler.cc:497] The NVIDIA driver's CUDA version is 11.4 which is older than the ptxas CUDA version (11.6.55). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [8]:
c = Pyfig(wandb_mode='disabled', debug=True) # online:on|disabled:off|offline:local, True: 
c.dict

<pyfig.Pyfig.data object at 0x7f3b147c1120> []
<pyfig.Pyfig.model object at 0x7f3b147c0fa0> []
<pyfig.Pyfig.opt object at 0x7f3b147c1090> []
<pyfig.Pyfig.sweep object at 0x7f3b147c1060> []
<pyfig.Pyfig.wandb object at 0x7f3b147c1030> []
<pyfig.Pyfig.slurm object at 0x7f3b147c0fd0> []
<pyfig.Pyfig object at 0x7f3b147c0f40> ['data', 'model', 'opt', 'sweep', 'wandb', 'slurm']
<class 'pyfig.Pyfig.data'>
this
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_ignore_attr', 'b_size', 'debug_print', 'dict', 'n_d', 'n_e', 'n_u', 'print']
this
['TMP', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '

NameError: name 'exit' is not defined

In [22]:
c.dict

{'TMP': PosixPath('/home/amawi/projects/hwat/exp/tmp'),
 'b_size': 16,
 'beta1': 0.9,
 'beta2': 0.99,
 'commit_id': '6e83b83',
 'cpus_per_task': 1,
 'data': <Recursion on dict with id=140132438273024>,
 'data_dir': PosixPath('/home/amawi/projects/data'),
 'debug': True,
 'dtype': 'f32',
 'entity': 'xmax1',
 'env': 'dex',
 'eps': 1e-08,
 'error': PosixPath('/home/amawi/projects/hwat/exp/tmp/e-%j.err'),
 'exp_id': 'FeUnvWt',
 'exp_name': 'junk',
 'exp_path': PosixPath('/home/amawi/projects/hwat/exp/junk/FeUnvWt'),
 'git_branch': 'main',
 'git_remote': 'origin',
 'gres': 'gpu:RTX3090:1',
 'half_precision': True,
 'iter_exp_dir': True,
 'job_name': 'junk',
 'job_type': 'training',
 'log_metric_step': 5,
 'log_sample_step': 5,
 'log_state_step': 10,
 'loss': 'l1',
 'lr': 0.001,
 'mail_type': 'FAIL',
 'method': 'random',
 'metrics': {'goal': 'minimize', 'name': 'validation_loss'},
 'model': <Recursion on dict with id=140132438273024>,
 'n_d': -7,
 'n_det': 1,
 'n_device': 1,
 'n_e': 7,
 'n_e

{'debug': True,
 'seed': 808017424,
 'project_root': PosixPath('/home/amawi/projects'),
 'project': 'hwat',
 'exp_name': 'junk',
 'data_dir': PosixPath('/home/amawi/projects/data'),
 'half_precision': True,
 'dtype': 'f32',
 'n_step': 1000,
 'b_size': 16,
 'n_e': 7,
 'n_u': 14,
 'n_d': -7,
 'data': {...},
 'n_sv': 16,
 'n_pv': 8,
 'n_fb': 2,
 'n_det': 1,
 'n_fb_out': 64,
 'model': {...},
 'optimizer': 'Adam',
 'beta1': 0.9,
 'beta2': 0.99,
 'eps': 1e-08,
 'lr': 0.001,
 'loss': 'l1',
 'opt': {...},
 'method': 'random',
 'name': 'sweep',
 'metrics': {'goal': 'minimize', 'name': 'validation_loss'},
 'parameters': {'batch_size': {'values': [16, 32, 64]},
  'epoch': {'values': [5, 10, 15]},
  'lr': {'max': 0.1, 'min': 0.0001}},
 'n_sweep': 10,
 'run_cap': 10,
 'sweep_id': '',
 'sweep': {...},
 'job_type': 'training',
 'entity': 'xmax1',
 'wandb': {...},
 'log_sample_step': 5,
 'log_metric_step': 5,
 'log_state_step': 10,
 'n_epoch': 20,
 'mail_type': 'FAIL',
 'partition': 'sm3090',
 'nodes'

In [2]:

from hwat import compute_s_perm
p_mask_u, p_mask_d = create_masks(c.data.n_e, c.data.n_u)
compute_s_perm: Callable = partial(compute_s_perm, p_mask_u=p_mask_u, p_mask_d=p_mask_d, n_u=c.data.n_u)

class FermiNet(nn.Module):
  
  n_sv: int
  n_pv: int
  n_fb: int
  n_fb_out: int
  n_det: int

  af_fb: Callable
  af_fb_out: Callable
  af_pseudo: Callable

  compute_s_emb: Callable
  compute_p_emb: Callable
  
  n_e: int
  n_u: int
  n_d: int 
  
  p_mask_u: jnp.ndarray 
  p_mask_d: jnp.ndarray
  
  @nn.compact
  def __call__(_i, x):
    
    n_e, n_u, n_d, n_det = _i.n_e, _i.n_u, n_d, _i.n_det

    x_s_var = _i.compute_s_emb(x)
    x_p_var = _i.compute_p_emb(x)

    x_s_res = x_p_res = 0.
    for _ in range(_i.n_fb):
        x_p_var = x_p_res = _i.af_fb(nn.Dense(_i.n_pv)(x_p_var)) + x_p_res
        
        x_s_var = _i.compute_s_perm(x_s_var, x_p_var)
        x_s_var = x_s_res = _i.af_fb(nn.Dense(_i.n_sv)(x_s_var)) + x_s_res

    x_w = jnp.concatentate([x_s_var, x_p_var], axis=-1)
    x_w = _i.af_fb_out(nn.Dense(_i.n_fb_out)(x_w))
    x_w = _i.af_pseudo(nn.Dense(n_det*n_e)(x_w))
    x_wu, x_wd = jnp.split(x_w, [n_u, n_d], axis=0)
    
    orb_u = jnp.split(x_wu * jnp.exp(-nn.Dense(n_u*n_det)), n_det, axis=1)
    orb_d = jnp.split(x_wd * jnp.exp(-nn.Dense(n_d*n_det)), n_det, axis=1)

    log_psi, sgn = logabssumdet(orb_u, orb_d)

    return log_psi

from utils import flat_any
from pprint import pprint
# import required modules
import inspect
import collections
args = inspect.signature(FermiNet.__init__).parameters.keys()
# print(args)
# pprint(flat_any(c.dict))
args = {k:v for k,v in flat_any(c.dict).items() if k in args}
# print(args)
model = FermiNet(**args)
rng_model = rnd.PRNGKey(1)
x = jnp.ones((c.data.n_e, 3))
params = model.init(rng_model, x)
model.apply(params, x)
# p_sample_step = jax.pmap(sample_step, axis_name='batch')



TypeError: FermiNet.__init__() missing 6 required positional arguments: 'n_fb_out', 'compute_s_emb', 'compute_p_emb', 'n_d', 'p_mask_u', and 'p_mask_d'

In [9]:
c.dict

{'debug': True,
 'seed': 808017424,
 'project_root': PosixPath('/home/amawi/projects'),
 'project': 'hwat',
 'exp_name': 'junk',
 'data_dir': PosixPath('/home/amawi/projects/data'),
 'half_precision': True,
 'dtype': 'f32',
 'n_step': 1000,
 'log_sample_step': 5,
 'log_metric_step': 5,
 'log_state_step': 10,
 'n_epoch': 20,
 'exp_id': 'caDCxM9',
 'iter_exp_dir': True,
 'server': 'svol.fysik.dtu.dk',
 'user': 'amawi',
 'entity': 'xmax1',
 'git_remote': 'origin',
 'git_branch': 'main',
 'env': 'dex',
 'sys_arg': ['--ip=127.0.0.1',
  '--stdin=9008',
  '--control=9006',
  '--hb=9005',
  '--Session.signature_scheme="hmac-sha256"',
  '--Session.key=b"b7c2890a-2305-4db6-bdb3-80f8891d6d13"',
  '--shell=9007',
  '--transport="tcp"',
  '--iopub=9009',
  '--f=/home/amawi/.local/share/jupyter/runtime/kernel-v2-20299JtvuDQDq849s.json'],
 'submit_state': -1,
 'submit': <function pyfig.Pyfig.submit(_i, sweep=False, commit_msg=None, commit_id=None)>,
 'data': {'debug': True, 'b_size': 16, 'n_e': 7, 'n

(self, model_hp: type, n_sv: int, n_pv: int, n_fb: int, n_fb_out: int, n_det: int, af_fb: Callable, af_fb_out: Callable, af_pseudo: Callable, compute_s_emb: Callable, compute_p_emb: Callable, n_e: int, n_u: int, n_d: int, p_mask_u: jax._src.basearray.Array, p_mask_d: jax._src.basearray.Array, pbc: bool = False, parent: Union[Type[ForwardRef('Module')], Type[flax.core.scope.Scope], Type[flax.linen.module._Sentinel], NoneType] = <flax.linen.module._Sentinel object at 0x7fcd550936d0>, name: Optional[str] = None) -> None


"(self, model_hp: type, n_sv: int, n_pv: int, n_fb: int, n_fb_out: int, n_det: int, af_fb: Callable, af_fb_out: Callable, af_pseudo: Callable, compute_s_emb: Callable, compute_p_emb: Callable, n_e: int, n_u: int, n_d: int, p_mask_u: jax._src.basearray.Array, p_mask_d: jax._src.basearray.Array, pbc: bool = False, parent: Union[Type[ForwardRef('Module')], Type[flax.core.scope.Scope], Type[flax.linen.module._Sentinel], NoneType] = <flax.linen.module._Sentinel object at 0x7fcd550936d0>, name: Optional[str] = None) -> None"

In [ ]:

# jax.tree_map(lambda x: x.shape, params) # Check the parameters

def create_train_state(c: Pyfig):
  model = CNN()
  params = model.init(c.rng_init, jnp.ones([1, 28, 28, 1]))['params']
  tx = optax.sgd(c.lr)
  return TrainState.create(apply_fn=model.apply, params=params, tx=tx)

state = create_train_state(c)
state = jax_utils.replicate(state)

@jax.jit
def train_step(params, state, b):
  
  b_energy = compute_energy(state)
  
  def loss_fn(p):
    model_out = state.apply_fn({'params':p}, b)
    log_psi, sgn = model_out
    return log_psi, sgn
  
  grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
  (log_psi, sgn), grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  
  variables = {
    'E batch'           : b_energy,
    r'$\log\psi$ batch' : log_psi,
    r'sgn$(\cdot)$'     : sgn
  }
  
  metrics = compute_metric(b_energy, log_psi, sgn)
  return state, metrics

train_step = jax.pmap(train_step, axis_name='batch')

In [ ]:
def copy_params_to_ema(state):
   state = state.replace(params_ema = state.params)
   return state

def apply_ema_decay(state, ema_decay):
    params_ema = jax.tree_map(lambda p_ema, p: p_ema * ema_decay + p * (1. - ema_decay), state.params_ema, state.params)
    state = state.replace(params_ema = params_ema)
    return state

p_apply_ema = jax.pmap(apply_ema_decay, in_axes=(0, None), axis_name='batch')
p_copy_params_to_ema = jax.pmap(copy_params_to_ema, axis_name='batch')


In [ ]:
from pprint import pprint
from pathlib import Path

def to_wandb_config(d, parent_key: str = '', sep: str ='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(to_wandb_config(v, new_key, sep=sep).items())
        else:
            if isinstance(v, Path):
                v = str(v)
            items.append((new_key, v))
    return dict(items)

wandb.init(
    job_type=c.wandb.job_type,
    entity=c.wandb.entity,
    project=c.project,
    config=to_wandb_config(c.dict),
    settings=wandb.Settings(start_method='fork'),  # idk why this is an issue
    dir=c.exp_path,
)

wandb.define_metric("*", step_metric="train/step")

# # Display a project workspace
# %wandb USERNAME/PROJECT
# # Display a single run
# %wandb USERNAME/PROJECT/runs/RUN_ID
# # Display a sweep
# %wandb USERNAME/PROJECT/sweeps/SWEEP_ID
# # Display a report
# %wandb USERNAME/PROJECT/reports/REPORT_ID
# # Specify the height of embedded iframe
# %wandb USERNAME/PROJECT -h 2048

In [ ]:
def get_data_stats():
    def normalize_to_neg_one_to_one(img):
        return img * 2 - 1  
    data_tr = datasets.FashionMNIST(
        root=c.data_dir, 
        download=True, 
        train=True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(28),
            transforms.CenterCrop(28),
        ]),
        target_transform = transforms.Compose([
        transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
    ])
    )
    imgs = torch.stack([img_t for img_t, _ in data_tr], dim=3)
    mean = imgs.view(1,-1).mean(dim=1) 
    std = imgs.view(1,-1).std(dim=1)
    print('Data mean: ', mean, 'Data std: ', std)
    return mean, std
mean, std = get_data_stats()
loader_tr, loader_test = get_fashion_loader(c.data.b_size, c.data_dir, mean=mean, std=std)  # is not an iterator or list
img, l = next(iter(loader_tr))

In [ ]:
import numpy as np
Image.fromarray(np.uint8((img[0, 0]*std+mean).cpu().numpy()*255))

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('1', size=(cols*w, rows*h))
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))

    return grid
std = float(std.cpu().numpy())
mean = float(mean.cpu().numpy())

In [ ]:
train_metrics = []

for ep in range(c.n_epoch):
    for step, (b, target) in enumerate(loader_tr):
        b = jnp.squeeze(jnp.expand_dims(jnp.array(b.numpy()), axis=(0,-1)), axis=2) # p, B, H, W, C
        rng, *train_step_rng = jax.random.split(rng, num=jax.local_device_count() + 1)
        train_step_rng = jnp.array(train_step_rng)
        
        state, metrics = p_train_step(train_step_rng, state, b)

        if step <= c.ema.update_after_step:
            state = p_copy_params_to_ema(state)

        elif step % c.ema.update_every == 0:
            ema_decay = ema_decay_fn(step)
            state =  p_apply_ema(state, ema_decay)

        if step % c.log_metric_step == 0:

            train_metrics.append(metrics)
            train_metrics = common_utils.get_metrics(train_metrics)
            
            summary = {
                f'train/{k}': v
                for k, v in jax.tree_map(lambda x: x.mean(), train_metrics).items()
            }

            train_metrics = []
            b = ((np.array(b)*std+mean)*255).astype(np.uint8)
            imgs = [wandb.Image(Image.fromarray(b[0, i].reshape(28, 28))) for i in range(9)]
            
            wandb.log({
                    "train/step": step, 
                    'train/sample': imgs,
                    **summary
            })
 
    print('Epoch: ', ep)

In [ ]:
# Image.fromarray(b[0, 0].reshape(28, 28, 1))
b[0, 0].max()